## Curbing Car Crashes in Chicago.

### Data understanding

#### Loading the data

In [60]:
df_crashes = pd.read_csv('data/Crashes.csv')
df_People = pd.read_csv('data/People.csv')
df_Vehicles = pd.read_csv('data/Vehicles.csv')


c:\condaa\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\condaa\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4,19,21,39,40,41,42,44,48,49,50,53,55,58,59,61,65,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


##### 1. Crashes data cleaning

In [61]:
df_crashes.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,JC199149,NaN,03/25/2019 02:43:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,1.0,2.0,0.0,14,2,3,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,JB422857,NaN,09/05/2018 08:40:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,...,0.0,0.0,2.0,0.0,8,4,9,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,JF318029,NaN,07/15/2022 12:45:00 AM,30,UNKNOWN,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,...,0.0,0.0,2.0,0.0,0,6,7,41.886336,-87.716203,POINT (-87.716203130599 41.886336409761)
3,05b1982cdba5d8a00e7e76ad1ecdab0e598429f78481d2...,JF378711,NaN,08/29/2022 11:30:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,11,2,8,41.749348,-87.721097,POINT (-87.721096727406 41.749348170421)
4,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,JF324552,NaN,07/15/2022 06:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,2.0,0.0,18,6,7,41.925111,-87.667997,POINT (-87.667997321599 41.925110815832)


We will go ahead and check for null values in our crashes data.

In [62]:
df_crashes.isna().sum()

CRASH_RECORD_ID                       0
RD_NO                              4598
CRASH_DATE_EST_I                 668460
CRASH_DATE                            0
POSTED_SPEED_LIMIT                    0
TRAFFIC_CONTROL_DEVICE                0
DEVICE_CONDITION                      0
WEATHER_CONDITION                     0
LIGHTING_CONDITION                    0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
LANE_CNT                         524138
ALIGNMENT                             0
ROADWAY_SURFACE_COND                  0
ROAD_DEFECT                           0
REPORT_TYPE                       20306
CRASH_TYPE                            0
INTERSECTION_RELATED_I           557272
NOT_RIGHT_OF_WAY_I               689379
HIT_AND_RUN_I                    498002
DAMAGE                                0
DATE_POLICE_NOTIFIED                  0
PRIM_CONTRIBUTORY_CAUSE               0
SEC_CONTRIBUTORY_CAUSE                0
STREET_NO                             0


We have to consider what data will be important for our modeling based on the problem domain and some of the columns will not really help us to generate insights frm our data. Therefore we will drop the columns below before proceeding to dealing the missing data. 

In [63]:
# Creating a function to remove irrelevant columns 
def ignore(data,columns_to_ignore):
    delete_cols = data.drop(columns_to_ignore,axis=1).copy()
    return delete_cols



In [64]:
# Identifying the columns to ignore from crashes data 
columns_to_ignore = ['REPORT_TYPE','DATE_POLICE_NOTIFIED', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE','LANE_CNT',
 'SEC_CONTRIBUTORY_CAUSE','STREET_NO','STREET_DIRECTION', 'STREET_NAME','PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 
'BEAT_OF_OCCURRENCE', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL','INJURIES_FATAL','INJURIES_INCAPACITATING', 
'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
'INJURIES_UNKNOWN']

# calling the function to drop the identified columns
df_crashes_clean = ignore(df_crashes,columns_to_ignore)

Some of the columns we have left have so many entries missing and therefore they will not be of help with the moddeling and therefore we will drop the entire columns. There are others like Longitude, Latitude and Location that have just a few missing entries and we will therefore only eliminat the missing rows.

In [65]:
df_crashes_clean.isna().sum()

CRASH_RECORD_ID                 0
POSTED_SPEED_LIMIT              0
TRAFFIC_CONTROL_DEVICE          0
DEVICE_CONDITION                0
WEATHER_CONDITION               0
LIGHTING_CONDITION              0
FIRST_CRASH_TYPE                0
TRAFFICWAY_TYPE                 0
ALIGNMENT                       0
ROADWAY_SURFACE_COND            0
ROAD_DEFECT                     0
CRASH_TYPE                      0
INTERSECTION_RELATED_I     557272
NOT_RIGHT_OF_WAY_I         689379
HIT_AND_RUN_I              498002
DAMAGE                          0
PRIM_CONTRIBUTORY_CAUSE         0
DOORING_I                  720954
WORK_ZONE_I                718930
WORK_ZONE_TYPE             719850
WORKERS_PRESENT_I          722055
NUM_UNITS                       0
CRASH_HOUR                      0
CRASH_DAY_OF_WEEK               0
CRASH_MONTH                     0
LATITUDE                     4679
LONGITUDE                    4679
LOCATION                     4679
dtype: int64

We will now go ahead and deal with the missing data from our data. Some of the columns have more than 50% of the data missing where some have less than 10% of the data missing.
Compared to total number of values  488,290, we have majority of values missing for 'Intersection_related', 'Not_right_of_way', 'Hit_and_run', 'Dooring', 'Work_zone', 'Work_Zone_Type', and 'Workers_Present'. This will therefore require us to remove these columns as well. 

In [66]:
# A function to drop missing data columns
def drop_missing(data,columns_to_ignore):
    delete_cols = data.drop(columns_to_ignore,axis=1).copy()
    return delete_cols


In [67]:
# Identifying the columns to drop
columns_missing = ['INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I','HIT_AND_RUN_I','DOORING_I','WORK_ZONE_I','WORK_ZONE_TYPE', 'WORKERS_PRESENT_I']
#calling the function do drop the missing data.
df_crashes_clean = drop_missing(df_crashes_clean,columns_missing)

Having dropped the missing data columns we will now drop the missing data rows that contained less than 10% of the data missing.

In [71]:
# Looking up the remaining missing data
df_crashes_clean.isna().sum()

CRASH_RECORD_ID               0
POSTED_SPEED_LIMIT            0
TRAFFIC_CONTROL_DEVICE        0
DEVICE_CONDITION              0
WEATHER_CONDITION             0
LIGHTING_CONDITION            0
FIRST_CRASH_TYPE              0
TRAFFICWAY_TYPE               0
ALIGNMENT                     0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
DAMAGE                        0
PRIM_CONTRIBUTORY_CAUSE       0
NUM_UNITS                     0
CRASH_HOUR                    0
CRASH_DAY_OF_WEEK             0
CRASH_MONTH                   0
LATITUDE                   4679
LONGITUDE                  4679
LOCATION                   4679
dtype: int64

We have eliminated the majority of the missing data . We want a clean data set therefore leaving no stone unturned we will go ahead to finaly dropna the remaining missing values.

In [73]:
# Dropping the remaining rows of missing data
df_crashes_clean.dropna(axis=0, inplace=True)

In [75]:
# Looking up the missing values.
df_crashes_clean.isna().sum()

CRASH_RECORD_ID            0
POSTED_SPEED_LIMIT         0
TRAFFIC_CONTROL_DEVICE     0
DEVICE_CONDITION           0
WEATHER_CONDITION          0
LIGHTING_CONDITION         0
FIRST_CRASH_TYPE           0
TRAFFICWAY_TYPE            0
ALIGNMENT                  0
ROADWAY_SURFACE_COND       0
ROAD_DEFECT                0
CRASH_TYPE                 0
DAMAGE                     0
PRIM_CONTRIBUTORY_CAUSE    0
NUM_UNITS                  0
CRASH_HOUR                 0
CRASH_DAY_OF_WEEK          0
CRASH_MONTH                0
LATITUDE                   0
LONGITUDE                  0
LOCATION                   0
dtype: int64

In [76]:
df_crashes_clean.shape

(718461, 21)

Success!!! We have dealt with the missing data as well as the columns to ignore. We started off with 49 columns down to 21 columns.

From this data set our target variable will be the primary contributor of crashes which helps us identify what caused the most crashes in chicago.

In [77]:
# Looking up our target variable
df_crashes_clean['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 277269
FAILING TO YIELD RIGHT-OF-WAY                                                        78544
FOLLOWING TOO CLOSELY                                                                71166
NOT APPLICABLE                                                                       37883
IMPROPER OVERTAKING/PASSING                                                          34912
FAILING TO REDUCE SPEED TO AVOID CRASH                                               30575
IMPROPER BACKING                                                                     28947
IMPROPER LANE USAGE                                                                  26137
IMPROPER TURNING/NO SIGNAL                                                           23751
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  23484
DISREGARDING TRAFFIC SIGNALS                                                         13912

From this we can identify that the majority of the crashes happen without the primary contributor being identified. This will not help us in curbing the crashes in chicago for we would like to get down to accidents root cause. Equally the 4th contributor of accidents is not applicable and equally should be dropped.

In [79]:
# Dropping Not applicable and unable to determine from our dataset
df_crashes_clean = df_crashes_clean[(df_crashes_clean.PRIM_CONTRIBUTORY_CAUSE != 'UNABLE TO DETERMINE') &
(df_crashes_clean.PRIM_CONTRIBUTORY_CAUSE != 'NOT APPLICABLE')].copy()

In [81]:
# Checking the number of rows and columns of our clean data set
df_crashes_clean.shape

(403309, 21)

##### 2. People data cleaning.

We will perfom a similar proces for our second data set so as to come up with a clean data set.

In [84]:
# displaying first five rows
df_People.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [89]:
#checking null values
df_People.isna().sum()

PERSON_ID                      0
PERSON_TYPE                    0
CRASH_RECORD_ID                0
RD_NO                      10529
VEHICLE_ID                 31009
CRASH_DATE                     0
SEAT_NO                  1264646
CITY                      428450
STATE                     412521
ZIPCODE                   526883
SEX                        25179
AGE                       461584
DRIVERS_LICENSE_STATE     653691
DRIVERS_LICENSE_CLASS     799546
SAFETY_EQUIPMENT            4473
AIRBAG_DEPLOYED            29812
EJECTION                   19354
INJURY_CLASSIFICATION        688
HOSPITAL                 1314559
EMS_AGENCY               1417344
EMS_RUN_NO               1557667
DRIVER_ACTION             323587
DRIVER_VISION             324040
PHYSICAL_CONDITION        322708
PEDPEDAL_ACTION          1555962
PEDPEDAL_VISIBILITY      1556020
PEDPEDAL_LOCATION        1555963
BAC_RESULT                322565
BAC_RESULT VALUE         1583513
CELL_PHONE_USE           1584214
dtype: int

Thats alot of missing data and it bascally helps us get rid of the columns that will not be of help in reducing the car crashes in chicago and thue we will first drop irrelevant columns. We will mainly remain with the columns that contain information that led to the accidents occurence rater that details about the drivers and dates.

In [96]:
#Identifying columns to drop
columns_to_ignore_2 = [ 'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'PERSON_ID', 'RD_NO', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 
'INJURY_CLASSIFICATION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO','ZIPCODE','PEDPEDAL_LOCATION','BAC_RESULT']

#calling function to drop the columns
df_People_clean = ignore(df_People,columns_to_ignore_2)

We will now handle the missing data from the remainin set of data

In [97]:
#Missing values
df_People_clean.isna().sum()

PERSON_TYPE                  0
CRASH_RECORD_ID              0
VEHICLE_ID               31009
SEX                      25179
AGE                     461584
DRIVER_ACTION           323587
DRIVER_VISION           324040
PHYSICAL_CONDITION      322708
PEDPEDAL_ACTION        1555962
PEDPEDAL_VISIBILITY    1556020
BAC_RESULT VALUE       1583513
CELL_PHONE_USE         1584214
dtype: int64

From this we can see that we have columns missing more than 1.5 million rows which translated to over 87% of the data. The rows are PEDPEDAL_ACTION, PEDPEDAL_VISIBILITY, BAC_RESULT VALUE, CELL_PHONE_USE.

In [98]:
# Identifying the columns to drop
columns_missing_2 = ['PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']


# Calling the function to drop missing values columns
df_People_clean = drop_missing(df_People_clean,columns_missing_2)

In [99]:
df_People_clean.isna().sum()

PERSON_TYPE                0
CRASH_RECORD_ID            0
VEHICLE_ID             31009
SEX                    25179
AGE                   461584
DRIVER_ACTION         323587
DRIVER_VISION         324040
PHYSICAL_CONDITION    322708
dtype: int64

Definately thats not all since we only had two columns without missing data and therefore we can not rush to dropna the remaining missing values since equaly they are many. We will look up the value counts first.

In [100]:
# Value_counts
df_People_clean.SEX.value_counts()


M    820006
F    597132
X    143055
Name: SEX, dtype: int64

In [106]:
# remove x from sex and set age to positive values
df_People_clean=df_People_clean[(df_People_clean['SEX'] != 'X')]
df_People_clean=df_People_clean[(df_People_clean['AGE'] > 0 )]

In [107]:
#Person value count
df_People_clean.PERSON_TYPE.value_counts()

DRIVER    892565
Name: PERSON_TYPE, dtype: int64

We only want data to do with the drivers thus remove the others.

In [108]:
# selecting driver alone
df_People_clean=df_People_clean[(df_People_clean['PERSON_TYPE'] == 'DRIVER')]

Having eliminated rows based on logic we will now look up the missing values.

In [110]:
#Missing values
df_People_clean.isna().sum()

PERSON_TYPE             0
CRASH_RECORD_ID         0
VEHICLE_ID            397
SEX                     0
AGE                     0
DRIVER_ACTION           0
DRIVER_VISION           0
PHYSICAL_CONDITION      0
dtype: int64

We now have a better data set and we will now drop the null values in the vihecle id column.

In [111]:
# Dropping the remaining rows of missing data
df_People_clean.dropna(axis=0, inplace=True)

In [112]:
#Missing values
df_People_clean.isna().sum()

PERSON_TYPE           0
CRASH_RECORD_ID       0
VEHICLE_ID            0
SEX                   0
AGE                   0
DRIVER_ACTION         0
DRIVER_VISION         0
PHYSICAL_CONDITION    0
dtype: int64

In [114]:
#shape of our remaining data
df_People_clean.shape

(892168, 8)

##### 3. Vehicles data cleaning 

Similarly we will clean this data set to obtain a useful dataset for our modelling.

In [116]:
# first five rows
df_Vehicles.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,1,DRIVER,NaN,1478881.0,NaN,FORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


72 columns!!! Thats a huge dataset which definately requires dropping of some columns which contain general information before we proceed to handling missing data.

In [118]:
# Dropping columns that we do not require
columns_to_ignore_3 = ['NUM_PASSENGERS', 'MAKE', 'MODEL', 'VEHICLE_YEAR', 'CMRC_VEH_I','LIC_PLATE_STATE','TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'TOWED_BY','TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I','AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I','AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I','CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO','COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE','CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO','HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO','MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I','MCS_VIO_CAUSE_CRASH_I', 
                       'IDOT_PERMIT_NO', 'WIDE_LOAD_I','MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS','TRAILER1_WIDTH', 'TRAILER2_WIDTH','CRASH_UNIT_ID', 'RD_NO', 'CRASH_DATE', 
                       'TRAILER1_LENGTH','TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG','CARGO_BODY_TYPE', 'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I'
                       ]
df_Vehicles_clean = ignore(df_Vehicles,columns_to_ignore_3)

In [119]:
#checking for null values
df_Vehicles_clean.isna().sum()

CRASH_RECORD_ID               0
UNIT_NO                       0
UNIT_TYPE                  2008
VEHICLE_ID                33157
VEHICLE_DEFECT            33157
VEHICLE_TYPE              33157
VEHICLE_USE               33157
TRAVEL_DIRECTION          33157
MANEUVER                  33157
EXCEED_SPEED_LIMIT_I    1471104
FIRST_CONTACT_POINT       36241
dtype: int64

What we have left is a smaller dataset with some missing values which require attention. We will have to do away with the column exceed speed limit which has so many missing values.

In [120]:
# column to drop
columnss = ['EXCEED_SPEED_LIMIT_I']
# call function to drop missing values
df_Vehicles_clean = drop_missing(df_Vehicles_clean,columnss)

We have six columns with the same amount of missing values and therefore since they contribute to a small percentage of the data we will drop the missing rows of data.

In [123]:
#droping null values
df_Vehicles_clean.dropna(axis=0, inplace=True)

We now have three clean data sets and for our modelling we will require to merge then into one data set using common columns between them.

#### Merging the datasets

We will now perfom inner joins between the datasets to have one dataset all together. The Vehicle and people datasets will be merged using the common column vihecle id and the merged datasets will further be merged to crashes dataset using crash record id.

In [128]:
# First merger
df_Vih_Pep = pd.merge(df_People_clean, df_Vehicles_clean, on=['VEHICLE_ID'], how = 'inner')


In [129]:
#first five columns
df_Vih_Pep.head()

,PERSON_TYPE,CRASH_RECORD_ID_x,VEHICLE_ID,SEX,AGE,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,CRASH_RECORD_ID_y,UNIT_NO,UNIT_TYPE,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,FIRST_CONTACT_POINT
0,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,M,25.0,UNKNOWN,UNKNOWN,UNKNOWN,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,1,DRIVER,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,FRONT
1,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,M,37.0,NONE,NOT OBSCURED,NORMAL,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,2,DRIVER,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,SIDE-RIGHT-FRONT
2,DRIVER,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,805348.0,F,34.0,UNKNOWN,NOT OBSCURED,NORMAL,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,1,DRIVER,UNKNOWN,PASSENGER,PERSONAL,NE,MERGING,FRONT-LEFT-CORNER
3,DRIVER,e9146986f4b0884d00ff3a54da5249263b4b36c15d01ce...,832624.0,M,53.0,UNKNOWN,UNKNOWN,NORMAL,e9146986f4b0884d00ff3a54da5249263b4b36c15d01ce...,1,DRIVER,UNKNOWN,BUS OVER 15 PASS.,CTA,E,TURNING LEFT,SIDE-LEFT
4,DRIVER,f15ccbd94a8e29ce8424882ce93061d4e1d0deb214acfe...,833984.0,M,65.0,UNKNOWN,UNKNOWN,NORMAL,f15ccbd94a8e29ce8424882ce93061d4e1d0deb214acfe...,1,DRIVER,UNKNOWN,PASSENGER,PERSONAL,S,STRAIGHT AHEAD,FRONT-RIGHT-CORNER


In [130]:
# crashes first five columns
df_crashes_clean.head()

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,...,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",IMPROPER TURNING/NO SIGNAL,2,14,2,3,41.884547,-87.641201,POINT (-87.64120093714 41.884547224337)
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,NO INJURY / DRIVE AWAY,"OVER $1,500","VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS,...",2,8,4,9,41.968562,-87.740659,POINT (-87.740659314632 41.968562453871)
3,05b1982cdba5d8a00e7e76ad1ecdab0e598429f78481d2...,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,DRY,...,NO INJURY / DRIVE AWAY,"$501 - $1,500",DISREGARDING TRAFFIC SIGNALS,2,11,2,8,41.749348,-87.721097,POINT (-87.721096727406 41.749348170421)
7,01aaa759c6bbefd0f584226fbd88bdc549de3ed1e46255...,40,NO CONTROLS,NO CONTROLS,CLOUDY/OVERCAST,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER LANE USAGE,2,17,6,7,41.975826,-87.650420,POINT (-87.650419778017 41.975826016449)
10,7b4700df143a2170b338fdf28c3ccb7c3aaa4527b0734e...,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,...,NO INJURY / DRIVE AWAY,"OVER $1,500",WEATHER,2,10,2,11,41.744152,-87.585945,POINT (-87.585945066953 41.744151639042)


We will need to rename the column on the first merger for the crashes Id so that we can now merge it to crashes dataset.

In [131]:
# Renaming a column
df_Vih_Pep.rename(columns={'CRASH_RECORD_ID_x': 'CRASH_RECORD_ID'}, inplace=True)

In [133]:
# Second merger
df_combined = pd.merge(df_Vih_Pep, df_crashes_clean, on=['CRASH_RECORD_ID'], how='inner')

In [135]:
# First five rows
df_combined.head()

,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,SEX,AGE,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,CRASH_RECORD_ID_y,UNIT_NO,...,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,M,37.0,NONE,NOT OBSCURED,NORMAL,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,2,...,NO INJURY / DRIVE AWAY,$500 OR LESS,IMPROPER OVERTAKING/PASSING,2,22,2,4,41.736044,-87.653404,POINT (-87.653404241798 41.736044089544)
1,DRIVER,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,805348.0,F,34.0,UNKNOWN,NOT OBSCURED,NORMAL,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,1,...,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER OVERTAKING/PASSING,2,18,1,2,41.997662,-87.700128,POINT (-87.700128203434 41.99766204922)
2,DRIVER,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,805338.0,M,36.0,UNKNOWN,NOT OBSCURED,NORMAL,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,2,...,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER OVERTAKING/PASSING,2,18,1,2,41.997662,-87.700128,POINT (-87.700128203434 41.99766204922)
3,DRIVER,49336aaca932f7935c361c892d31e01360a08133079e3a...,834553.0,M,26.0,IMPROPER BACKING,WINDSHIELD (WATER/ICE),NORMAL,49336aaca932f7935c361c892d31e01360a08133079e3a...,1,...,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER BACKING,2,12,3,5,41.931414,-87.759020,POINT (-87.759019512841 41.931414096945)
4,DRIVER,49336aaca932f7935c361c892d31e01360a08133079e3a...,834560.0,F,64.0,NONE,NOT OBSCURED,NORMAL,49336aaca932f7935c361c892d31e01360a08133079e3a...,2,...,NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER BACKING,2,12,3,5,41.931414,-87.759020,POINT (-87.759019512841 41.931414096945)


In [136]:
df_combined.shape

(577689, 37)

Having cleaned and merged our data we have a total of 577689 rows and 37 columns for the combined dataset.